In [ ]:
import requests
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# this method fetch data from the backend; triggered by url call
# fetch data allows you to qurey from the body argument

def fetch_data(body, index_name):
    url = 'http://127.0.0.1:9090/query-es'
    response = requests.post(url, json={'body': body, 'index': index_name})
    if response.status_code == 200:
        return response.json()
    else:
        return f"Failed to fetch data: Status code {response.status_code}"


# this function return the WHOLE es index
# be careful with large data set
# 中文再说一遍，这个function把整个表都读进来，如果数据很大会很慢
# 为了避免内存爆炸，考虑使用上面的函数写query

def fetch_all_data(index: str):
    url = 'http://127.0.0.1:9090/fetch-all'
    response = requests.post(url, json={'index': index}, headers={"Content-Type": "application/json"})
    if response.status_code == 200:
        return response.json()  # Make sure to return the parsed JSON
    else:
        return f"Failed to fetch data: Status code {response.status_code}"



In [ ]:
# testing querying the first 5 rows from the sensor index
query = {"size": 10, "query": {"match_all": {}}}
index = 'sensor'
test = fetch_data(index)

In [ ]:
test

In [ ]:
# reading the fix data into pd.dataframe
import pandas as pd
def es_into_df(es_outcome: list):
    data = []
    for document in es_outcome:
        if '_source' in document:
            data.append(document['_source'])
    df = pd.DataFrame(data)
    return df

In [ ]:
es_into_df(test)